<a href="https://colab.research.google.com/github/kundajelab/label_shift_experiments/blob/master/diabetic_retinopathy/DiabeticRetinopathyDomainAdaptation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://zenodo.org/record/3412990/files/kaggle_dr.txt.gz?download=1 -O kaggle_dr.txt.gz
!wget https://zenodo.org/record/3412990/files/messidor_dr.txt.gz?download=1 -O messidor_dr.txt.gz

--2019-09-17 00:48:24--  https://zenodo.org/record/3412990/files/kaggle_dr.txt.gz?download=1
Resolving zenodo.org (zenodo.org)... 188.184.65.20
Connecting to zenodo.org (zenodo.org)|188.184.65.20|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 106643 (104K) [application/octet-stream]
Saving to: ‘kaggle_dr.txt.gz’

kaggle_dr.txt.gz    100%[===================>] 104.14K   297KB/s    in 0.4s    

2019-09-17 00:48:26 (297 KB/s) - ‘kaggle_dr.txt.gz’ saved [106643/106643]

--2019-09-17 00:48:26--  https://zenodo.org/record/3412990/files/messidor_dr.txt.gz?download=1
Resolving zenodo.org (zenodo.org)... 188.184.65.20
Connecting to zenodo.org (zenodo.org)|188.184.65.20|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 38723 (38K) [application/octet-stream]
Saving to: ‘messidor_dr.txt.gz’

messidor_dr.txt.gz  100%[===================>]  37.82K  --.-KB/s    in 0.1s    

2019-09-17 00:48:27 (321 KB/s) - ‘messidor_dr.txt.gz’ saved [38723/38723]



In [0]:
import gzip
import numpy as np

orig_kaggle_onehot_labels = [] #5 classes
orig_kaggle_predlogits = []
for row in gzip.open("kaggle_dr.txt.gz"):
    _, kaggle_label, logits = row.decode("utf-8").rstrip().split("\t")
    kaggle_label = int(kaggle_label)
    logits = [float(x) for x in logits.split(",")]
    labelsvec = np.zeros(5)
    labelsvec[kaggle_label] = 1.0
    orig_kaggle_onehot_labels.append(labelsvec)
    orig_kaggle_predlogits.append(logits)
orig_kaggle_onehot_labels = np.array(orig_kaggle_onehot_labels)
orig_kaggle_predlogits = np.array(orig_kaggle_predlogits)

orig_messidor_onehot_labels = [] #4 classes
orig_messidor_predlogits = []
for row in gzip.open("messidor_dr.txt.gz"):
    _, messidor_label, logits = row.decode("utf-8").rstrip().split("\t")
    messidor_label = int(messidor_label)
    logits = [float(x) for x in logits.split(",")]
    labelsvec = np.zeros(4)
    labelsvec[messidor_label] = 1.0
    orig_messidor_onehot_labels.append(labelsvec)
    orig_messidor_predlogits.append(logits)
orig_messidor_onehot_labels = np.array(orig_messidor_onehot_labels)
orig_messidor_predlogits = np.array(orig_messidor_predlogits)

In [4]:
from sklearn.metrics import roc_auc_score
from scipy.special import softmax

print(roc_auc_score(y_true=1-orig_kaggle_onehot_labels[:,0],
                    y_score=1-softmax(orig_kaggle_predlogits, axis=-1)[:,0]))
print(roc_auc_score(y_true=1-orig_messidor_onehot_labels[:,0],
                    y_score=1-softmax(orig_messidor_predlogits, axis=-1)[:,0]))

0.8945151902722942
0.8920633498604387


In [5]:
![[ -e abstention ]] || git clone https://github.com/blindauth/abstention
%cd /content/abstention
!git checkout iterativebias
!git pull
!pip uninstall abstention
!pip install .
%cd ..

Cloning into 'abstention'...
remote: Enumerating objects: 56, done.
remote: Counting objects: 100% (56/56), done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 278 (delta 39), reused 38 (delta 21), pack-reused 222
Receiving objects: 100% (278/278), 86.26 KiB | 3.92 MiB/s, done.
Resolving deltas: 100% (181/181), done.
/content/abstention
Branch 'iterativebias' set up to track remote branch 'iterativebias' from 'origin'.
Switched to a new branch 'iterativebias'
Already up to date.
Processing /content/abstention
  Created wheel for abstention: filename=abstention-0.1.2.1-cp36-none-any.whl size=45664 sha256=80d137cc6ad8c994d780b085928fc61716869d97733c2b18d44370b9b8abd2e3
  Stored in directory: /tmp/pip-ephem-wheel-cache-xrzij5wa/wheels/60/8e/d2/9e9ca02e7b5f76bfda2e2daa6dcbe42c19095c502ccb653729
Successfully built abstention
/content


In [0]:
#do calibration and domain adaptation
from collections import OrderedDict
from importlib import reload
import abstention
reload(abstention)
reload(abstention.calibration)
reload(abstention.label_shift)
from abstention.calibration import (
    TempScaling, CrossValidatedBCTS, ConfusionMatrix, softmax)
from abstention.label_shift import (EMImbalanceAdapter, BBSEImbalanceAdapter)

calibname_to_calibfactory = OrderedDict([
    ('None', abstention.calibration.Softmax()),
    ('TS', TempScaling(verbose=False)),
    ('BCTS', TempScaling(verbose=False,
                         bias_positions=[0,1,2,3,4])),
    #('CVBCTS', CrossValidatedBCTS(verbose=True,
    #                              num_crossvalidation_splits=5,
    #                              betas_to_try=[0.0, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1.0, 10]
    #                             ))
])

adaptname_to_adapter = OrderedDict([
    ('em', EMImbalanceAdapter()),
    ('bbse-hard', BBSEImbalanceAdapter(soft=False)),
    ('bbse-soft', BBSEImbalanceAdapter(soft=True))
])
    
adaptncalib_pairs = [
    ('bbse-hard', 'None'),
    ('bbse-soft', 'None'),
    ('bbse-soft', 'TS'),
    ('bbse-soft', 'BCTS'),
    #('bbse-soft', 'CVBCTS'),
    ('em', 'None'),
    ('em', 'TS'),
    ('em', 'BCTS'),
    #('em', 'CVBCTS'),
]
    

In [0]:
def get_diseased_v_not_accuracy(preds, onehot_labels):
    return np.mean((np.argmax(preds,axis=-1) > 0.0)
                   ==(1-onehot_labels[:,0]))

def get_frac_compatible_preds_messidor(kaggletype_preds,
                                       messidortype_onehot_labels):
    total_compatible_preds = 0
    argmax_kaggle_preds = np.argmax(kaggletype_preds, axis=-1)
    argmax_messidor_labels = np.argmax(messidortype_onehot_labels, axis=-1)
    for the_kaggle_pred, the_messidor_label in zip(argmax_kaggle_preds,
                                                   argmax_messidor_labels):
        if (the_kaggle_pred==0 and the_messidor_label==0):
            compatible=True
        elif (the_kaggle_pred==1 and the_messidor_label==1):
            compatible=True
        elif ((the_kaggle_pred==1 or the_kaggle_pred==2)
              and the_messidor_label==2):
            compatible=True
        elif ( (the_kaggle_pred >= 2) and the_messidor_label==3 ):
            compatible=True
        else:
            compatible=False
        if (compatible):
            total_compatible_preds += 1
    return total_compatible_preds/len(kaggletype_preds)
  
  
def convert_kagglepreds_to_messidortype(kaggletype_preds):
  to_return = np.concatenate(
          [kaggletype_preds[:,0:1],
           kaggletype_preds[:,1:2], 
           (kaggletype_preds[:,2:3]
            +kaggletype_preds[:,3:4]
            +kaggletype_preds[:,4:5])], axis=1)
  #normalize to sum to 1
  return to_return/np.sum(to_return, axis=-1)[:,None]
  
  
def get_cohens_quadratic_kappa_dropping_class2(kaggletype_preds,
                                               messidortype_onehot_labels):
    filtered_kaggletype_preds =\
      convert_kagglepreds_to_messidortype(kaggletype_preds)[
        messidortype_onehot_labels[:,2]==0]
    filtered_messidortype_onehot_labels =\
      messidortype_onehot_labels[messidortype_onehot_labels[:,2]==0][:,[0,1,3]]
    return abstention.abstention.weighted_kappa_metric(
            predprobs=filtered_kaggletype_preds,
            true_labels=filtered_messidortype_onehot_labels,
            weights=np.array([[0,1,4], [1,0,1], [4,1,0]]),
            mode='argmax')

In [0]:
from collections import defaultdict

metric_to_adaptname_to_perfs = defaultdict(lambda: defaultdict(list))

for seed in [10,20,30,40,50,60,70,80,100]:
  
  bootstrap_messidor_indices = np.random.RandomState(seed).choice(
      a=np.arange(len(orig_messidor_onehot_labels)),
      size=len(orig_messidor_onehot_labels))  
  bootstrap_kaggle_indices = np.random.RandomState(seed).choice(
      a=np.arange(len(orig_kaggle_onehot_labels)),
      size=len(orig_kaggle_onehot_labels))
  
  messidor_onehot_labels = orig_messidor_onehot_labels[bootstrap_messidor_indices]
  kaggle_onehot_labels = orig_kaggle_onehot_labels[bootstrap_kaggle_indices]
  messidor_predlogits = orig_messidor_predlogits[bootstrap_messidor_indices]
  kaggle_predlogits = orig_kaggle_predlogits[bootstrap_kaggle_indices]
  
  calibname_to_calibfunc = OrderedDict()
  calibname_to_calibkaggle = OrderedDict()
  calibname_to_calibmessidor = OrderedDict()
  for calibname in calibname_to_calibfactory:
      calibfactory = calibname_to_calibfactory[calibname]
      calibfunc = calibfactory(valid_preacts=kaggle_predlogits,
                               valid_labels=kaggle_onehot_labels)
      calibname_to_calibfunc[calibname] = calibfunc
      calibname_to_calibkaggle[calibname] = calibfunc(kaggle_predlogits)
      calibname_to_calibmessidor[calibname] = calibfunc(messidor_predlogits)

  adaptncalib_to_preds = OrderedDict()
  for (adaptname, calibname) in adaptncalib_pairs:
      calibkaggle_preds = calibname_to_calibkaggle[calibname]
      calibmessidor_preds = calibname_to_calibmessidor[calibname]
      adapter = adaptname_to_adapter[adaptname]
      adapter_func = adapter(valid_labels=kaggle_onehot_labels,
                             tofit_initial_posterior_probs=calibmessidor_preds,
                             valid_posterior_probs=calibkaggle_preds)
      adapted_messidor_preds = adapter_func(calibmessidor_preds)
      adaptncalib_to_preds[adaptname+":"+calibname] = adapted_messidor_preds
  
  for calibname in calibname_to_calibmessidor:
    #print("Calibname", calibname)
    calibkagglepreds = calibname_to_calibkaggle[calibname]
    calibmessidorpreds = calibname_to_calibmessidor[calibname]
    #print("kaggle class proportion diff",
    #      (np.mean(calibkagglepreds, axis=0)-
    #       np.mean(kaggle_onehot_labels, axis=0)) )
    #print("kaggle accuracy for diseased v not",
    #      get_diseased_v_not_accuracy(preds=calibkagglepreds,
    #                                  onehot_labels=kaggle_onehot_labels))
    
    #print("Messidor diseased v not class proportion diff",
    #      (np.mean(calibmessidorpreds[:,0], axis=0)-
    #       np.mean(messidor_onehot_labels[:,0], axis=0))) 
    messidorformat_calibmessidorpreds_noclass2 =\
      convert_kagglepreds_to_messidortype(kaggletype_preds=calibmessidorpreds)[
        messidor_onehot_labels[:,2]==0]
    messidor_onehot_labels_noclass2 = messidor_onehot_labels[:,[0,1,3]][
        messidor_onehot_labels[:,2]==0]
    #print("Messidor no class 2 proportion diff",
    #      (np.mean(messidorformat_calibmessidorpreds_noclass2, axis=0)-
    #       np.mean(messidor_onehot_labels_noclass2, axis=0)))
    
    #print("messidor accuracy for diseased v not",
    #      get_diseased_v_not_accuracy(preds=calibmessidorpreds,
    #                                  onehot_labels=messidor_onehot_labels))
    #print("messidor total compatible preds",
    #      get_frac_compatible_preds_messidor(
    #          kaggletype_preds=calibmessidorpreds,
    #          messidortype_onehot_labels=messidor_onehot_labels))
    #print("quadratic kappa:", get_cohens_quadratic_kappa_dropping_class2(
    #        kaggletype_preds=calibmessidorpreds,
    #        messidortype_onehot_labels=messidor_onehot_labels))
  
  for adaptncalib in adaptncalib_to_preds:
      #print(adaptncalib,"Accuracy for diseased v not",
      #      get_diseased_v_not_accuracy(preds=messidor_preds,
      #                                  onehot_labels=messidor_onehot_labels))
      messidor_preds = adaptncalib_to_preds[adaptncalib]
      metric_to_adaptname_to_perfs["totalcompatiblepreds"][adaptncalib].append(
          get_frac_compatible_preds_messidor(
                  kaggletype_preds=messidor_preds,
                  messidortype_onehot_labels=messidor_onehot_labels))

      metric_to_adaptname_to_perfs["quadratickappa"][adaptncalib].append(
          get_cohens_quadratic_kappa_dropping_class2(
                kaggletype_preds=messidor_preds,
                messidortype_onehot_labels=messidor_onehot_labels))
      #print(adaptncalib,"Messidor diseased v not class proportion diff",
      #      (np.mean(messidor_preds[:,0], axis=0)-
      #       np.mean(messidor_onehot_labels[:,0], axis=0))) 
      #messidorformat_messidorpreds_noclass2 =\
      #  convert_kagglepreds_to_messidortype(kaggletype_preds=messidor_preds)[
      #    messidor_onehot_labels[:,2]==0]
      #messidor_onehot_labels_noclass2 = messidor_onehot_labels[:,[0,1,3]][
      #    messidor_onehot_labels[:,2]==0]
      #print(adaptncalib,"Messidor no class 2 proportion diff",
      #      (np.mean(messidorformat_messidorpreds_noclass2, axis=0)-
      #       np.mean(messidor_onehot_labels_noclass2, axis=0)))


In [10]:
for metric in ["quadratickappa", "totalcompatiblepreds"]:
  print("Metric",metric)
  for adaptncalib in adaptncalib_to_preds:
    print(adaptncalib, np.mean(metric_to_adaptname_to_perfs[metric][adaptncalib]))

Metric quadratickappa
bbse-hard:None 0.8118530204222877
bbse-soft:None 0.8252134605260347
bbse-soft:TS 0.8122614069014431
bbse-soft:BCTS 0.821721245227683
em:None 0.8256087128465137
em:TS 0.8183388709140387
em:BCTS 0.8393002114503503
Metric totalcompatiblepreds
bbse-hard:None 0.765
bbse-soft:None 0.7573148148148149
bbse-soft:TS 0.7472222222222222
bbse-soft:BCTS 0.7737037037037038
em:None 0.7591666666666667
em:TS 0.7519444444444444
em:BCTS 0.7664814814814815
